# Point cloud registration with Easy Open3D
In this tutorial we will use the **Easy Open3D** package to register Suzanne, the monkey head mascot from [Blender](https://www.blender.org), to (1) a complete scene point cloud where she is sitting on a chair and (2) a partial point cloud of the same scene in form of an RGB-D image. The result will be her pose, i.e. her rotation and translation in world coordinates in this scene.
![](./tests/test_data/test_data.png)

In [ ]:
# Import package functionality
from easy_o3d import utils
from easy_o3d.registration import IterativeClosestPoint, FastGlobalRegistration, RANSAC, ICPTypes

## 1. Registration on complete data
### 1.1 Load and visualize point clouds
Let's start by loading the `source` (Suzanne) and `target` (the scene) point clouds from file. Both are stored in the PLY file format as meshes, so we need to sample points from the mesh surfaces to obtain point clouds. We will also load the known ground truth pose in world coordinates to evaluate the fit of the registration results.

In [ ]:
# Load ground truth pose. It is stored in a JSON file which must contain keys
# containing "rot" and "trans" substrings.
gt_path = "tests/test_data/ground_truth_pose.json"
gt_pose = utils.eval_transformation_data(gt_path)

# Load source and target meshes from PLY files and sample points from their surface.
source_path = "tests/test_data/suzanne.ply"
source = utils.eval_data(data=source_path, number_of_points=10000)

target_path = "tests/test_data/suzanne_on_chair.ply"
target = utils.eval_data(data=target_path, number_of_points=100000)

# Visualize data
utils.draw_geometries(geometries=[source, target])

### 1.2 Process point clouds
Before we run the registration algorithms, it is almost always a good idea to pre-process the data. Through voxel downsampling, we can reduce the computation time and facilitate the finding of correspondences between source and target. We also compute FPFH feature which are needed for the global registration (initialization) phase.

In [ ]:
source_down = utils.process_point_cloud(point_cloud=source,
                                        downsample=utils.DownsampleTypes.VOXEL,
                                        downsample_factor=0.01)

_, source_feature = utils.process_point_cloud(point_cloud=source_down,
                                              compute_feature=True,
                                              search_param_knn=100,
                                              search_param_radius=0.05)

target_down = utils.process_point_cloud(point_cloud=target,
                                        downsample=utils.DownsampleTypes.VOXEL,
                                        downsample_factor=0.01)

_, target_feature = utils.process_point_cloud(point_cloud=target_down,
                                              compute_feature=True,
                                              search_param_knn=100,
                                              search_param_radius=0.05)

# Visualize data
utils.draw_geometries(geometries=[source_down, target_down])

### 1.3 Initialization: RANSAC
Registration algorithms are commonly categorized into _global_ and _local_, because finding an initial match and an accurate one comes with a different set of challenges. Here we will use the **RANSAC** global registration algorithm to find an initial pose, which we will subsequently refine.

In [ ]:
ransac = RANSAC()
ransac_result = ransac.run(source=source_down,
                           target=target_down,
                           source_feature=source_feature,
                           target_feature=target_feature,
                           draw=True,
                           overwrite_colors=True)

error_rot, error_trans = utils.get_transformation_error(ransac_result.transformation, gt_pose)
print("Error rotation [deg]:", error_rot, "Error translation [m]:", error_trans)
print("Rotation estimate:")
print(ransac_result.transformation[:3, :3])
print("Translation estimate:", ransac_result.transformation[:3, 3])
print("Runtime [s]:", ransac_result.runtime)

### 1.4 Initialization: Fast Global Registration
The second supported initializer is **Fast Global Registration** which was developed more recently and promises to be faster than RANSAC or more precise, if given the same time.

In [ ]:
fgr = FastGlobalRegistration()
fgr_result = fgr.run(source=source_down,
                     target=target_down,
                     source_feature=source_feature,
                     target_feature=target_feature,
                     draw=True,
                     overwrite_colors=True)

error_rot, error_trans = utils.get_transformation_error(fgr_result.transformation, gt_pose)
print("Error rotation [deg]:", error_rot, "Error translation [m]:", error_trans)
print("Rotation estimate:")
print(fgr_result.transformation[:3, :3])
print("Translation estimate:", fgr_result.transformation[:3, 3])
print("Runtime [s]:", fgr_result.runtime)

Because initialization can be sensitive to external factors, we can simply run it multiple times and pick the best result:

In [ ]:
fgr = FastGlobalRegistration()
fgr_result = fgr.run_n_times(source=source_down,
                             target=target_down,
                             source_feature=source_feature,
                             target_feature=target_feature,
                             n_times=3)

error_rot, error_trans = utils.get_transformation_error(fgr_result.transformation, gt_pose)
print("Error rotation [deg]:", error_rot, "Error translation [m]:", error_trans)
print("Rotation estimate:")
print(fgr_result.transformation[:3, :3])
print("Translation estimate:", fgr_result.transformation[:3, 3])
print("Runtime [s]:", fgr_result.runtime)

### 1.5 Refinement: Iterative Closest Point
Now that we have an initial guess of the pose of Suzanne (which is already very good for this toy example), we can refine it using the **Iterative Closest Point** algorithm. There are multiple variants and we will make use of the _point-to-plane_ formulation because it tends to be more accurate than _point-to-point_ (albeit sometimes less robust) and we already have computed the required point normals as prerequesits of the FPFH features.

In [ ]:
icp = IterativeClosestPoint(estimation_method=ICPTypes.PLANE)
result_icp = icp.run(source=source_down,
                     target=target_down,
                     init=ransac_result.transformation,  # Plug in our initial estimate
                     draw=True,
                     overwrite_colors=True)

error_rot, error_trans = utils.get_transformation_error(result_icp.transformation, gt_pose)
print("Error rotation [deg]:", error_rot, "Error translation [m]:", error_trans)
print("Rotation estimate:")
print(result_icp.transformation[:3, :3])
print("Translation estimate:", result_icp.transformation[:3, 3])
print("Runtime [s]:", result_icp.runtime)

The quality of the registration result depends on a number of parameters, most notably the _maximum correspondence distance_, determining which distance a point pair in the source and target point cloud is allowed to have at most to be concidered as correspondence. We can alleviate this hyperparameter problem somewhat by running the algorithm with multiple correspondence distances in sequence from large to small.

In [ ]:
# Here we use the original source and target point clouds as they will be downsampled
# during multi-scale registration.
icp = IterativeClosestPoint(estimation_method=ICPTypes.PLANE)
result_icp = icp.run_multi_scale(source=source,  
                                 target=target,
                                 init=ransac_result.transformation,
                                 draw=True,
                                 overwrite_colors=True)

error_rot, error_trans = utils.get_transformation_error(result_icp.transformation, gt_pose)
print("Error rotation [deg]:", error_rot, "Error translation [m]:", error_trans)
print("Rotation estimate:")
print(result_icp.transformation[:3, :3])
print("Translation estimate:", result_icp.transformation[:3, 3])
print("Runtime [s]:", result_icp.runtime)

## 2. Changing the target: RGB-D images
Let's now move to the more relatistic (and challenging) scenario of partial target point clouds in the form of RGB-D images. RGB-D images can be mapped from 2D to 3D space using the intrinsic camera parameters and transformed into the world coordinate frame using the extrinsic camera parameters.

### 2.1 Load camera parameters from BlenderProc BopWriter
[BlenderProc](https://github.com/DLR-RM/BlenderProc) is a an awesome open source synthetic training data generator based on Blender targeted at machine learning applications. It was used to generate the synthetic RGB-D test data for the Easy Open3D package. It can store the generated data in the [Bop Challenge](https://bop.felk.cvut.cz/home) data format, for which various readers are implemented in Easy Open3D for convenience.

In [ ]:
scene_id = 20
path_to_scene_camera_json = "tests/test_data/bop_data/obj_of_interest/train_pbr/000000/scene_camera.json"
path_to_camera_json = "tests/test_data/bop_data/obj_of_interest/camera.json"
output_path = "tests/test_data"

camera_parameters = utils.get_camera_parameters_from_blenderproc_bopwriter(path_to_scene_camera_json,
                                                                           path_to_camera_json,
                                                                           scene_id)[0]

### 2.2 Load, process and visualize the RGB-D image
Having obtained the intrinsic and extrinsic camera parameters of the scene, we can now convert the RGB-D image to a colored point cloud. The partial scene point cloud is much more challenging to solve so we can make use of the [`hyperopt`](https://github.com/hummat/easy-o3d/blob/master/scripts/hyperopt.py) script to find good set of hyperparameters for processing and registration. Due to the large number of hyperparameters and their interaction, this is quite challenging and tedious to do by hand.

In [ ]:
# Path to color and depth image
color = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/rgb/{str(scene_id).zfill(6)}.png"
depth = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/depth/{str(scene_id).zfill(6)}.png"

# Load the RGB-D image and convert it to a point cloud
target_rgbd = utils.eval_data(data=[color, depth],
                              camera_intrinsic=camera_parameters.intrinsic,
                              camera_extrinsic=camera_parameters.extrinsic,
                              depth_trunc=3.0)  # Truncate depth at 3m

# Downsample, remove outlier, estimate normals
target_rgbd = utils.process_point_cloud(point_cloud=target_rgbd,
                                        downsample=utils.DownsampleTypes.VOXEL,
                                        downsample_factor=0.007579519638230087,
                                        remove_outlier=utils.OutlierTypes.STATISTICAL,
                                        outlier_std_ratio=3.0,
                                        estimate_normals=True,
                                        search_param=utils.SearchParamTypes.RADIUS,
                                        search_param_radius=0.012467595584023949)

_, target_rgbd_feature = utils.process_point_cloud(point_cloud=target_rgbd,
                                                   compute_feature=True,
                                                   search_param=utils.SearchParamTypes.RADIUS,
                                                   search_param_radius=0.06968346743904905)

# Process source again to correspond to target voxel size
source_path = "tests/test_data/suzanne.ply"
source = utils.eval_data(data=source_path, number_of_points=8586)

source_down = utils.process_point_cloud(point_cloud=source,
                                        downsample=utils.DownsampleTypes.VOXEL,
                                        downsample_factor=0.008829352302399583,
                                        estimate_normals=True,
                                        recalculate_normals=True,
                                        search_param=utils.SearchParamTypes.KNN,
                                        search_param_knn=47)

_, source_feature = utils.process_point_cloud(point_cloud=source_down,
                                              compute_feature=True,
                                              search_param=utils.SearchParamTypes.RADIUS,
                                              search_param_radius=0.06968346743904905)

utils.draw_geometries(geometries=[source_down, target_rgbd])

### 2.3 Initialization: RANSAC
We use the RANSAC algorithm to find an initial pose estimate using the best hyperparameters found during optimization. Still, this initialization can somtetimes fail. In this case, either use the `run_n_times` functionality or simply rerun this cell.

In [ ]:
ransac = RANSAC(max_iteration = 107323,
                max_correspondence_distance=0.029175728431991857,
                confidence = 0.9,
                similarity_threshold = 0.7)
ransac_result = ransac.run(source=source_down,
                           target=target_rgbd,
                           source_feature=source_feature,
                           target_feature=target_rgbd_feature,
                           draw=True,
                           overwrite_colors=True)

error_rot, error_trans = utils.get_transformation_error(ransac_result.transformation, gt_pose)
print("Error rotation [deg]:", error_rot, "Error translation [m]:", error_trans)
print("Rotation estimate:")
print(ransac_result.transformation[:3, :3])
print("Translation estimate:", ransac_result.transformation[:3, 3])
print("Runtime [s]:", ransac_result.runtime)

### 2.4 Refinement: Iterative Closest Point
To make sure the ICP algorithm doesn't get confused we can crop the target around the initial source position. This will fail if the initial position is extremely inaccurate, though ICP will fail in such a case anyway.

In [ ]:
icp = IterativeClosestPoint(max_iteration=100,
                            max_correspondence_distance=0.01)
result_icp = icp.run(source=source_down,
                     target=target_rgbd,
                     init=ransac_result.transformation,
                     crop_target_around_source=True,
                     crop_scale=1.5,
                     draw=True,
                     overwrite_colors=True)

error_rot, error_trans = utils.get_transformation_error(result_icp.transformation, gt_pose)
print("Error rotation [deg]:", error_rot, "Error translation [m]:", error_trans)
print("Rotation estimate:")
print(result_icp.transformation[:3, :3])
print("Translation estimate:", result_icp.transformation[:3, 3])
print("Runtime [s]:", result_icp.runtime)

In [ ]:
# Visualize the final pose again using the complete and colore scene
icp.draw_registration_result(source=source, target=target_rgbd, pose=result_icp.transformation)

## Wrapping up
This concludes our little tutorial. There is still a lot of functionality that didn't make it in so feel free to have a look around.